# 1. Data Pre-processing

In [8]:
import numpy as np
import re 

In [15]:
import pandas as pd

df = pd.read_csv('Conversational_Pos_Politeness_LIWC_data.csv')

df.rename(columns={'A':'Text Content',
                   'B':'Document',
                   'C':'Codes',
                   'D':'author_association',
                   'E':'is_poster',
                   'F':'quote_len',
                   'G':'percent_len_thread',
                   'H':'percent_len_comment',
                   'I':'percent_pos_thread',
                   'J':'percent_pos_comment',
                   'K':'is_first_comment',
                   'L':'is_last_comment',
                   'M':'percent_start_time_thread',
                   'N':'percent_end_time_thread',
                   'O':'percent_previous_time_thread',
                   'P':'percent_next_time_thread',
                   'Q':'contain_code',
                   'R':'pos_str',
                   'S':'polite_score'}, inplace=True)

In [16]:
# check if any row contains NaN
df1 = df[df.isnull().any(axis=1)]
print(df1.shape[0])

0


In [17]:
display(df)

,Text Content,Document,Codes,author_association,is_poster,quote_len,percent_len_thread,percent_len_comment,percent_pos_thread,percent_pos_comment,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,Is it possible to add feature in order to forc...,issue9388_allCommentsText.doc,"9388_claim 1, 9388_support 1",NOT FOUND,True,152,0.750000,1.000000,0.014035,0.800000,...,3.45,0.00,0.00,3.45,0.00,0.00,0.00,0.00,0.00,0.00
1,This can be very useful when there is a lot of...,issue9388_allCommentsText.doc,"9388_warrant 1, 9388_support 1",NOT FOUND,True,95,0.500000,0.666667,0.017544,1.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,"I'm using VS Code VERSION_NUM Insider, but I ...",issue9388_allCommentsText.doc,9388_non-argumentative,CONTRIBUTOR,False,153,0.650000,1.000000,0.021053,1.000000,...,3.33,3.33,3.33,0.00,0.00,6.67,0.00,3.33,0.00,6.67
3,otherwise there is no reason for double click,issue9388_allCommentsText.doc,"9388_warrant 1, 9388_support 1",NONE,True,45,0.200000,0.888889,0.028070,1.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,SCREEN_NAME The 'preview' function does not k...,issue9388_allCommentsText.doc,"9388_ground 1, 9388_against 1",CONTRIBUTOR,False,190,0.825000,1.000000,0.031579,0.500000,...,2.94,0.00,0.00,0.00,0.00,0.00,0.00,5.88,5.88,2.94
5,"Not sure what you are going for here, why woul...",issue9388_allCommentsText.doc,"9388_against 1, 9388_warrant 1",CONTRIBUTOR,False,127,0.625000,0.757576,0.035088,1.000000,...,3.85,0.00,0.00,3.85,0.00,3.85,0.00,0.00,7.69,0.00
6,I also am trying to figure out how to do this,issue9388_allCommentsText.doc,"9388_warrant 1, 9388_support 1",NONE,False,45,0.275000,0.392857,0.038596,0.066667,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,because I want the explorer panel to have focu...,issue9388_allCommentsText.doc,"9388_warrant 1, 9388_support 1",NONE,False,131,0.700000,1.000000,0.045614,0.200000,...,3.57,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,I was hoping I could type the name of a file a...,issue9388_allCommentsText.doc,"9388_warrant 1, 9388_support 1",NONE,False,113,0.600000,0.857143,0.059649,0.466667,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.17,0.00,0.00
9,even with KB focus in the explorer window it d...,issue9388_allCommentsText.doc,"9388_ground 1, 9388_support 1",NONE,False,61,0.300000,0.428571,0.066667,0.600000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.33,0.00,0.00


In [18]:
def flag_category(text, flag):
    if flag == 'argu_part':
        if text.find('claim') > -1:
            return 1
        elif text.find('warrant') > -1:
            return 2
        elif text.find('ground') > -1:
            return 3
        else:
            return 0
    elif flag =='argu_count':
        argu_count = 0
        if text.find('claim') > -1:
            argu_count += 1
        if text.find('warrant') > -1:
            argu_count += 1
        if text.find('ground') > -1:
            argu_count += 1  
        return argu_count
    else:
        return text.find(flag) > -1


In [19]:
df['is_non_argumentative'] = df.apply(lambda x: flag_category(x['Codes'], 'non-argumentative'), axis=1)
df['is_against'] = df.apply(lambda x: flag_category(x['Codes'], 'against'), axis=1)
df['argu_part'] = df.apply(lambda x: flag_category(x['Codes'], 'argu_part'), axis=1)
df['argu_count'] = df.apply(lambda x: flag_category(x['Codes'], 'argu_count'), axis=1)

## 1.2. Check if argu_part is assigned correctly
Quotation take one and only one of 'claim', 'warrant' or 'data'


In [20]:
# this dataframe needs to be empty
df[(df.is_non_argumentative == False) & (df.argu_part == 0)]

,Text Content,Document,Codes,author_association,is_poster,quote_len,percent_len_thread,percent_len_comment,percent_pos_thread,percent_pos_comment,...,Exclam,Dash,Quote,Apostro,Parenth,OtherP,is_non_argumentative,is_against,argu_part,argu_count


In [21]:
# this dataframe needs to be empty
df[(df.is_non_argumentative == False) & (df.argu_count > 1)]

,Text Content,Document,Codes,author_association,is_poster,quote_len,percent_len_thread,percent_len_comment,percent_pos_thread,percent_pos_comment,...,Exclam,Dash,Quote,Apostro,Parenth,OtherP,is_non_argumentative,is_against,argu_part,argu_count


## 1.3. Write to file

In [22]:
# remove argu_count column, so it does not write to the file
df = df.drop(['argu_count'], axis=1)

In [23]:
df.to_csv('Preprocessed.csv', index=False)